In [4]:
import random

class CasinoGame:
    def __init__(self):
        self.deck = [2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 10, 11] * 4  # Simplified Blackjack deck

    def play_round(self, bot_action_fn):
        """Simulate one round of Blackjack."""
        hand = [random.choice(self.deck), random.choice(self.deck)]
        dealer_card = random.choice(self.deck)
        chips = 100  # Starting chips

        while sum(hand) < 21:
            # Get bot's action from neural network
            state = [sum(hand), dealer_card, chips]
            action_probs = bot_action_fn(state)
            action = np.argmax(action_probs)

            if action == 0:  # Hit
                hand.append(random.choice(self.deck))
                if sum(hand) > 21:
                    chips -= 10
                    break
            elif action == 1:  # Stand
                # Dealer plays
                dealer_hand = [dealer_card, random.choice(self.deck)]
                while sum(dealer_hand) < 17:
                    dealer_hand.append(random.choice(self.deck))
                if sum(dealer_hand) > 21 or sum(hand) > sum(dealer_hand):
                    chips += 10
                else:
                    chips -= 10
                break
        return chips

In [5]:
import neat

def eval_genomes(genomes, config):
    game = CasinoGame()
    for genome_id, genome in genomes:
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        genome.fitness = 0

        # Play 10 rounds to evaluate fitness
        for _ in range(10):
            # Define a lambda to pass state to the neural network
            action_fn = lambda state: net.activate(state)
            chips = game.play_round(action_fn)
            genome.fitness += chips

In [7]:
def run_neat():
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         'config.txt')
    population = neat.Population(config)
    stats = neat.StatisticsReporter()
    population.add_reporter(stats)
    population.add_reporter(neat.StdOutReporter(True))

    winner = population.run(eval_genomes, 50)  # Evolve for 50 generations
    print("Best bot evolved!")

if __name__ == "__main__":
    run_neat()

Exception: No such config file: c:\Users\astha\OneDrive\Desktop\nuralcasinoapp\config.txt